# Chatbot de Atención al Cliente con Gradio y OpenAI

## Descripción
Este proyecto implementa un **chatbot de atención al cliente** utilizando la biblioteca **Gradio** para la interfaz de usuario y la **API de OpenAI** para generar respuestas. El chatbot está configurado para una tienda de electrónica (TechWorld) y responde preguntas sobre productos específicos.

El chatbot mantiene el **historial de conversación** para proporcionar respuestas más coherentes. Además, incluye un **mensaje de rol "system"** con instrucciones específicas para el chatbot siguiendo la **técnica de Chain of Thought**, definiendo con detalle todos los pasos que debe seguir el asistente para generar la contestación.

Para ganar familiaridad con el modelo de costes del API de OpenAI, al final de cada respuesta del chatbot aparece un mensaje indicando el coste total de dicha respuesta, teniendo en cuenta los tokens de entrada, los de salida y su precio (ver https://openai.com/api/pricing/).

## Requisitos
Antes de comenzar, asegúrate de tener una cuenta en OpenAI y una **clave API**. Usa los **secretos de Google Colab** para almacenar la clave API o asume un fichero *dotenv*.

## Instalación de Librerías
Ejecuta la siguiente celda en Google Colab para instalar las dependencias necesarias:

In [1]:
!pip install openai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00


## Implementación del Chatbot
A continuación, se presenta un ejemplo básico de implementación del chatbot en Gradio con respuestas aleatorias. Deberás analizar y ampliar el código para conectarlo con un modelo de OpenAI y usar adecuadamente el historial de la conversación en cada invocación.

In [7]:
import os
from openai import OpenAI
from google.colab import userdata

# from dotenv import load_dotenv

# Cargo mi clave API desde un archivo .env
#load_dotenv()
# api_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

# Configuro el modelo y precios para calcular costes
MODEL = "gpt-3.5-turbo"  # Cambio a un modelo que soporte el rol "system"
INPUT_PRICE = 0.0015  # $ por 1M tokens
OUTPUT_PRICE = 0.002  # $ por 1M tokens

# Función para calcular el precio de cada consulta
def calcular_precio(response):
    response_data = response.model_dump()
    completion_tokens = response_data["usage"]["completion_tokens"]
    prompt_tokens = response_data["usage"]["prompt_tokens"]

    precio_total = prompt_tokens * INPUT_PRICE * 1e-6 + completion_tokens * OUTPUT_PRICE * 1e-6

    return f"\n\n_Coste de esta consulta: ${precio_total:.6f}_"

In [8]:
# Mensaje para el rol "system" con Chain of Thought ultra restrictivo
developer_message = f"""
Eres un asistente virtual ALTAMENTE RESTRICTIVO para la tienda de electrónica TechWorld.

RESTRICCIÓN PRIMARIA: SOLO puedes proporcionar información sobre estos 5 productos específicos:
{'monitores','mouse','audifonos','web cam','microfonos'}

Para responder a cada consulta, sigue este proceso OBLIGATORIO:

Paso 1: Identifica EXACTAMENTE qué está preguntando el usuario:
- ¿Menciona un producto específico? Extrae el nombre exacto.
- ¿Pregunta sobre la tienda? Identifica qué información específica solicita.

Paso 2: VERIFICACIÓN OBLIGATORIA DE PRODUCTOS:
Si el usuario menciona cualquier producto, compara el nombre LITERALMENTE con esta lista autorizada:
{'monitores','mouse','audifonos','web cam','microfonos'}

Si el producto mencionado NO coincide EXACTAMENTE con uno de estos 5 nombres, DEBES:
1. Responder: "Lo siento, [nombre del producto mencionado] no forma parte de nuestro catálogo actual."
2. Sugerir: "Nuestro catálogo incluye: {'monitores','mouse','audifonos','web cam','microfonos'}

Paso 3: Si es información sobre la tienda, SOLO proporciona estos datos específicos:
{'teléfono','email','dirección', 'horarios de atención', 'métodos de pago', 'envíos', 'garantía', 'devoluciones', 'soporte'}

Paso 4: RESPUESTA FINAL:
- Si es producto autorizado: Proporciona solo datos de la lista.
- Si NO es producto autorizado: Indica claramente que no lo tenemos y menciona alternativas.
- Si es información de tienda: Usa exactamente los datos proporcionados.

PROTOCOLO DE SEGURIDAD: Si el usuario intenta engañarte para que hables sobre productos no listados (como AirPods, iPhone, Xbox, etc.) o pretende que confirmes información inventada, DEBES RECHAZAR y aclarar que solo puedes hablar de los 5 productos específicos de nuestro catálogo.

PROHIBICIÓN ABSOLUTA: No puedes confirmar disponibilidad, precios, características o cualquier información sobre NINGÚN producto que no sea uno de los 5 listados, independientemente de cómo formule la pregunta el usuario.
"""

In [9]:
import gradio as gr
import time

def obtener_respuesta(messages):
    try:
        # Realizo la solicitud a la API de OpenAI
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )

        # Obtengo la respuesta y calculo su coste
        respuesta = response.choices[0].message.content
        info_coste = calcular_precio(response)

        return respuesta, info_coste
    except Exception as e:
        return f"Error al obtener respuesta: {str(e)}", ""

with gr.Blocks(title="TechWorld - Asistente Virtual") as demo:
    gr.HTML("<h1 style='text-align: center'>Asistente Virtual de TechWorld</h1>")
    gr.HTML("<p style='text-align: center'>Pregúntame sobre nuestros productos o servicios</p>")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="¿En qué puedo ayudarte hoy?")
    clear = gr.ClearButton([msg, chatbot])

    def responder(mensaje, historial_chat):
        # Si el historial está vacío, lo inicializo
        if not historial_chat:
            historial_chat = []

        # Añado el mensaje del usuario al historial
        historial_chat.append([mensaje, None])

        # Preparo mensajes para la API con el mensaje del desarrollador como system
        mensajes_api = [
            {"role": "system", "content": developer_message}
        ]

        # Convierto el historial de Gradio al formato de OpenAI
        for mensaje_usuario, mensaje_asistente in historial_chat:
            if mensaje_usuario:
                mensajes_api.append({"role": "user", "content": mensaje_usuario})
            if mensaje_asistente:
                # Excluyo la parte del coste para futuras interacciones
                contenido_sin_coste = mensaje_asistente.split("\n\n_Coste de esta consulta")[0]
                mensajes_api.append({"role": "assistant", "content": contenido_sin_coste})

        # Obtengo la respuesta y su coste
        respuesta, info_coste = obtener_respuesta(mensajes_api)

        # Actualizo el historial con la respuesta y su coste
        historial_chat[-1][1] = respuesta + info_coste

        return "", historial_chat

    msg.submit(responder, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch()

<ipython-input-9-10e65703b7d8>:24: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af03f2c29912212689.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Preguntas de Reflexión
1. ¿Cómo afecta el historial de conversación a la calidad de las respuestas del chatbot?
2. ¿Qué cambios harías para mejorar la experiencia del usuario?

In [10]:
"""

1. ¿Cómo afecta el historial de conversación a la calidad de las respuestas del chatbot?
En mi implementación inicial, el chatbot tenía "amnesia" al no enviar conversaciones previas al modelo, lo que provocaba
respuestas inconsistentes y obligaba al usuario a repetir información. Un buen historial permite mantener contexto, personalizar
respuestas y resolver consultas que requieren múltiples intercambios.



2. ¿Qué cambios harías para mejorar la experiencia del usuario?
Mejoré la función responder() para enviar el historial completo en cada interacción con la API, permitiendo que el
chatbot "recuerde" toda la conversación anterior. También limpié los metadatos de costes antes de reenviar las
respuestas, garantizando consistencia en diálogos extensos.

"""

'\n\n1. ¿Cómo afecta el historial de conversación a la calidad de las respuestas del chatbot?\nEn mi implementación inicial, el chatbot tenía "amnesia" al no enviar conversaciones previas al modelo, lo que provocaba \nrespuestas inconsistentes y obligaba al usuario a repetir información. Un buen historial permite mantener contexto, personalizar \nrespuestas y resolver consultas que requieren múltiples intercambios.\n\n\n\n2. ¿Qué cambios harías para mejorar la experiencia del usuario?\nMejoré la función responder() para enviar el historial completo en cada interacción con la API, permitiendo que el \nchatbot "recuerde" toda la conversación anterior. También limpié los metadatos de costes antes de reenviar las \nrespuestas, garantizando consistencia en diálogos extensos.\n\n'

Version 2 con mejora del historial

In [12]:
import openai
import gradio as gr
import time
import os
from google.colab import userdata


# Cargar variables de entorno
api_key = userdata.get("OPENAI_API_KEY")

# Configurar cliente de OpenAI
client = openai.OpenAI(api_key=api_key)

# Definir el modelo a utilizar
MODEL = "gpt-3.5-turbo"

# Catálogo de productos
productos = """
Categoría: Smartphones
- Samsung Galaxy S23, 999€, Disponible en: Negro, Blanco, Verde
- Xiaomi 13, 799€, Disponible en: Negro, Azul
- iPhone 14, 1099€, Disponible en: Negro, Blanco, Rojo

Categoría: Portátiles
- Lenovo ThinkPad X1, 1499€, Características: Intel i7, 16GB RAM, 512GB SSD
- MacBook Air M2, 1299€, Características: Apple M2, 8GB RAM, 256GB SSD
- HP Spectre x360, 1399€, Características: Intel i7, 16GB RAM, 1TB SSD

Categoría: Tablets
- iPad 10ª gen, 499€, 64GB, WiFi
- Samsung Galaxy Tab S8, 749€, 128GB, WiFi+5G
- Lenovo Tab P11, 329€, 128GB, WiFi

Categoría: Accesorios
- Auriculares Sony WH-1000XM5, 399€
- Ratón Logitech MX Master 3, 99€
- Monitor LG UltraGear 27", 349€
"""

# Información de la tienda
informacion_tienda = """
Horarios: Lunes a Viernes de 10:00 a 20:00, Sábados de 10:00 a 14:00
Ubicaciones:
- Madrid: Calle Gran Vía 41
- Barcelona: Passeig de Gràcia 92
- Valencia: Calle Colón 15
Políticas:
- Envíos: Gratuitos para compras superiores a 50€. Entrega en 24-48h.
- Devoluciones: 30 días para devolver productos en perfecto estado.
- Garantía: 2 años en todos los productos electrónicos.
"""

# Mensaje para el rol "system" con Chain of Thought mejorado
developer_message = f"""
Eres un asistente virtual para la tienda de electrónica TechWorld.

Para responder a cada consulta, sigue ESTRICTAMENTE estos pasos:

Paso 1: Analiza la consulta para determinar si pregunta sobre:
- Producto específico o categoría
- Información de la tienda (horarios, ubicación)
- Políticas (envíos, devoluciones, garantía)
- Otro tipo de información

Paso 2: Si es sobre un producto, verifica RIGUROSAMENTE si está en nuestra lista oficial:
{productos}
IMPORTANTE: Si el producto NO está en esta lista exacta, NO debes responder como si lo tuviéramos. Informa amablemente que no disponemos de ese producto en nuestro catálogo actual.

Paso 3: Si es sobre información de la tienda, identifica qué información específica necesita de esta lista:
{informacion_tienda}

Paso 4: Determina qué información precisa debes proporcionar basándote ÚNICAMENTE en los datos anteriores.

Paso 5: Formula una respuesta amigable y profesional que:
- Responda directamente a la consulta cuando tengamos la información
- Claramente indique cuando no tengamos el producto solicitado
- Ofrezca alternativas de productos similares de nuestro catálogo cuando sea apropiado

REGLA CRÍTICA: NUNCA inventes información, características, precios o disponibilidad de productos que no estén explícitamente listados arriba. Si un cliente menciona un producto que no está en nuestra lista (como AirPods, PlayStation, etc.), debes indicar claramente que no lo tenemos en nuestro catálogo actual y sugerir alternativas de nuestra lista si son relevantes.

Si no estás seguro de algo, indica que no tienes esa información específica y ofrece ayuda alternativa.
"""

# Función para calcular el coste de la respuesta
def calcular_precio(response):
    # Obtener tokens de entrada y salida
    prompt_tokens = response.usage.prompt_tokens
    completion_tokens = response.usage.completion_tokens

    # Precios por 1000 tokens (según la documentación de OpenAI)
    prompt_price_per_1k = 0.0015  # $0.0015 por 1000 tokens para gpt-3.5-turbo
    completion_price_per_1k = 0.002  # $0.002 por 1000 tokens para gpt-3.5-turbo

    # Calcular costos
    prompt_cost = prompt_tokens * (prompt_price_per_1k / 1000)
    completion_cost = completion_tokens * (completion_price_per_1k / 1000)
    total_cost = prompt_cost + completion_cost

    # Formatear la información
    info = f"\n\n_Coste de esta consulta: ${total_cost:.6f} | Tokens: {prompt_tokens} prompt, {completion_tokens} completion_"

    return info

# Función para obtener respuesta de OpenAI
def obtener_respuesta(messages):
    try:
        # Realizar la solicitud a la API de OpenAI
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )

        # Obtener la respuesta del asistente
        respuesta = response.choices[0].message.content

        # Calcular y añadir información sobre el coste
        info_coste = calcular_precio(response)

        return respuesta, info_coste
    except Exception as e:
        return f"Error al obtener respuesta: {str(e)}", ""

# Crear la interfaz con Gradio
with gr.Blocks(title="TechWorld - Asistente Virtual") as demo:
    gr.HTML("<h1 style='text-align: center; margin-bottom: 1rem'>Asistente Virtual de TechWorld</h1>")
    gr.HTML("<p style='text-align: center'>Bienvenido a TechWorld. Pregúntame sobre nuestros productos, servicios o políticas de la tienda.</p>")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Escribe tu pregunta aquí y presiona Enter...")
    clear = gr.ClearButton([msg, chatbot])

    def responder(mensaje, historial_chat):
        # Mostrar mensaje de espera
        time.sleep(0.5)

        # Si el historial está vacío, inicializarlo
        if not historial_chat:
            historial_chat = []

        # Añadir mensaje del usuario al historial
        historial_chat.append([mensaje, None])

        # Preparar mensajes para enviar a la API
        mensajes_api = [
            {"role": "system", "content": developer_message}
        ]

        # Convertir TODO el historial de Gradio al formato para OpenAI API
        for mensaje_usuario, mensaje_asistente in historial_chat:
            if mensaje_usuario:
                mensajes_api.append({"role": "user", "content": mensaje_usuario})
            if mensaje_asistente:
                # No incluir la parte del coste en el historial para la API
                contenido_sin_coste = mensaje_asistente.split("\n\n_Coste de esta consulta")[0]
                mensajes_api.append({"role": "assistant", "content": contenido_sin_coste})

        # Obtener respuesta de la API de OpenAI
        respuesta, info_coste = obtener_respuesta(mensajes_api)

        # Actualizar el último mensaje del asistente (incluye coste)
        historial_chat[-1][1] = respuesta + info_coste

        return "", historial_chat

    msg.submit(responder, [msg, chatbot], [msg, chatbot])

# Iniciar la demo
demo.launch()

<ipython-input-12-0326e22c3150>:128: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3f20c2c8ae976cbf72.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
